# Load goodreads data
Og lidt oprydning.

In [248]:

import main
import pandas as pd
# load goodreads data, clean whitespace from author names and filter out unread books
df = main.read_data()
df['Author'] = df['Author'].str.replace(r'\s+', ' ', regex=True)
df['Author'].unique()
df=df[df['Read Count'] > 0]


# collapse the list on author and add book count and average 'Average Rating'
df = df.groupby('Author').apply(lambda x: pd.Series({
    'Title': ', '.join(x['Title']),
    'Average Rating': x['Average Rating'].mean(),
})).reset_index()


df['Books'] = df['Title'].str.split(', ')
df.drop('Title', axis=1, inplace=True)


C:\Users\christian.dalager\AppData\Local\Temp\ipykernel_33328\3162756380.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Author').apply(lambda x: pd.Series({


# Author country lookup


## Funktion til at kalde wiki datas SPARQL endpoint


In [249]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
import re
from SPARQLWrapper import SPARQLWrapper, JSON


def execute_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s (christian@dalager.com)" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


def get_author_countries(author_name):
    author_name = re.sub(r"\s+", " ", author_name)
    author_name = author_name.strip()

    endpoint_url = "https://query.wikidata.org/sparql"

    query = (
        """SELECT ?countryLabel WHERE {
        ?author ?label "%s"@en.
        ?author wdt:P27 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }"""
        % author_name
    )

    results = execute_query(endpoint_url, query)

    bindings = results["results"]["bindings"]
    countries = list({entry["countryLabel"]["value"] for entry in bindings})

    return countries


country = get_author_countries("Lars Kepler")
print(country)

# country['countryLab']['value']

[]


## Kald data apiet

In [211]:
# extract all author names from "Author" to a unique list
import pandas as pd
authors = df['Author'].unique()

print(f'Requesting {len(authors)} authors')

# Create a list to collect author and their countries
author_country_list = []

# Iterate over all authors and get the countries  (1.5 min)
for author in authors:
    countries = get_author_countries(author)  # Make sure this returns a list of countries
    author_country_list.append({'Author': author, 'Countries': countries})

df_authorcountries = pd.DataFrame(author_country_list)
    
# save to csv
df_authorcountries.to_csv('../data/author_countries.csv', index=False)


# Begynd at arbejde med listerne

In [ ]:
df_authorcountries = pd.read_csv('../data/author_countries.csv')

import ast
# handle authors with no countries
for index, row in df_authorcountries.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_authorcountries.at[index, 'Countries'] = countries


# Manually located authors' countries in the dataset

In [234]:


df_manually_placed = pd.read_csv('../data/manually_placed.csv')
# import ast
# # handle authors with no countries
for index, row in df_manually_placed.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_manually_placed.at[index, 'Countries'] = countries

df_manually_placed.rename(columns={'Countries': 'Countries_Manually_Placed'}, inplace=True)
merged_df = pd.merge(df_authorcountries, df_manually_placed[['Author', 'Countries_Manually_Placed']], on='Author', how='left', validate='1:1')
# Replace the 'Countries' column with the corrected countries where available
merged_df['Countries'] = merged_df['Countries_Manually_Placed'].fillna(merged_df['Countries'])

# Drop the 'Countries_Manually_Placed' column if you don't need it anymore
merged_df=merged_df.drop(columns=['Countries_Manually_Placed'])

df_authorcountries = merged_df

# Complete the final dataset

In [238]:


# read author countries
#df_authorcountries = pd.read_csv('../data/author_countries.csv')
#df_authorcountries

df_final = pd.merge(df, df_authorcountries, on='Author', how='left', validate='1:1')


df_final.to_csv('../data/myauthorlist_with_countries_and_books.csv', index=False)









# Show me the countries!

In [247]:
# list of uniqe countries



cs=list(df_final['Countries'])
cs = {item for sublist in cs for item in sublist}

cs = list(cs)

cs.sort()
cs




['Ancient Rome',
 'Australia',
 'Austria',
 'Austria-Hungary',
 'Brazil',
 'Canada',
 'Chile',
 'Cisleithania',
 'Classical Athens',
 'Colombia',
 'Czech Republic',
 'Czechoslovakia',
 'Denmark',
 'Dutch Republic',
 'England',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'German Empire',
 'Germany',
 'Greenland',
 'Haiti',
 'Hungary',
 'India',
 'Iran',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Kingdom of Denmark',
 'Kingdom of England',
 'Kingdom of Great Britain',
 'Kingdom of Italy',
 'Kingdom of Scotland',
 'Kingdom of the Netherlands',
 'Lebanon',
 'Malaysia',
 'Mexico',
 'Nazi Germany',
 'New Zealand',
 'Nigeria',
 'Northern Ireland',
 'Norway',
 "People's Republic of China",
 'Poland',
 'Republic of Ireland',
 'Russia',
 'Russian Empire',
 'Russian Socialist Federative Soviet Republic',
 'Russian Soviet Federative Socialist Republic',
 'Scotland',
 'Senegal',
 'Somalia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'United Kingdom of Great Bri